In [2]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv('dataset/broden1_224/index.csv')

df.head()

,image,split,ih,iw,sh,sw,color,object,part,material,scene,texture
0,opensurfaces/25605.jpg,train,224,224,112,112,opensurfaces/25605_color.png,NaN,NaN,opensurfaces/25605_material.png,NaN,NaN
1,opensurfaces/24183.jpg,train,224,224,112,112,opensurfaces/24183_color.png,NaN,NaN,opensurfaces/24183_material.png,NaN,NaN
2,opensurfaces/72705.jpg,train,224,224,112,112,opensurfaces/72705_color.png,NaN,NaN,opensurfaces/72705_material.png,NaN,NaN
3,opensurfaces/100509.jpg,train,224,224,112,112,opensurfaces/100509_color.png,NaN,NaN,opensurfaces/100509_material.png,NaN,NaN
4,ade20k/ADE_train_00003891.jpg,train,224,224,112,112,ade20k/ADE_train_00003891_color.png,ade20k/ADE_train_00003891_object.png,NaN,NaN,83.0,NaN


In [3]:
import pandas as pd
import numpy as np

# 设置随机种子以确保结果可复现
np.random.seed(42)

# 加载数据
data = pd.read_csv('dataset/broden1_224/index.csv')

# 删除场景列中的缺失值，计算每个场景的频率
scene_counts = data['scene'].dropna().value_counts()
total_images = len(data['scene'].dropna())
proportions = scene_counts / total_images

# 为大约1,000张图片的数据集计算需要的样本数
target_sample_size_1000 = 1000
samples_needed_1000 = (proportions * target_sample_size_1000).round().astype(int)
adjustment_1000 = target_sample_size_1000 - samples_needed_1000.sum()
samples_needed_1000.iloc[0] += adjustment_1000

# 为大约10,000张图片的数据集计算需要的样本数
target_sample_size_10000 = 10000
samples_needed_10000 = (proportions * target_sample_size_10000).round().astype(int)
adjustment_10000 = target_sample_size_10000 - samples_needed_10000.sum()
samples_needed_10000.iloc[0] += adjustment_10000

# 定义一个执行分层抽样的函数
def stratified_sampling(data, category_counts):
    sampled_data = pd.DataFrame()
    for category, count in category_counts.items():
        category_data = data[data['scene'] == category]
        sampled_data = pd.concat([sampled_data, category_data.sample(min(len(category_data), count), random_state=42)])
    return sampled_data

# 执行抽样
sampled_data_1000 = stratified_sampling(data, samples_needed_1000)
sampled_data_10000 = stratified_sampling(data, samples_needed_10000)

# 保存数据到CSV文件
sampled_data_1000.to_csv('dataset/broden1_224/index_s_1000.csv', index=False)
sampled_data_10000.to_csv('dataset/broden1_224/index_b_10000.csv', index=False)